# PROVIDE A RANDOMLY DISTRIBUTED DELAY TO ACTIVITIES

In [1]:
# Import dependencies
import datetime
import dateutil
from openclsim.appendix import get_event_log
import openclsim.model as model
import openclsim.plugins as plugins
import pandas as pd
import scipy.stats as st
import simpy

In [58]:
# Create a simulation environment
class SimulationEnvironment(object):
    """ A OpenCLSim simulation object.

    Use the class methods to define the activities and start the
    simulation. In this example, a basic activity is modelled using 
    the `model.BasicActivity` class and the length of the activity is
    extended using a random delay with the `model.HasDelayPlugin`. For
    further reference, check the `DelayBasicActivity` class below.

    Parameters
    ----------
        start_date: datetime.datetime
            A datetime object specifying the simulation start date.
    """


    def __init__(self, start_date: datetime.datetime):

        # Define start date and time of simulation
        self.start_date = start_date
        start_utc = start_date.replace(tzinfo=dateutil.tz.UTC)
        start_epoch = start_utc.timestamp()

        # Initialise a SimPy simulation environment
        self.env = simpy.Environment(initial_time=start_epoch)

    def define_activities(self):
        """ Use this function to define your activities. """

        # In this example we define a basic activity
        length = 3600 * 5  # The activity takes about 5 hours.
        delay_by = st.uniform(loc=0, scale=1)  # The delay as RANDOM variable.
        
        activity = DelayedBasicActivity(
            env=self.env,  # The SimPy environment
            duration=length,  # The length the activity takes
            registry={} ,  # 
            name='Delayed basic activity',  # Description
            delay_percentage=delay_by,  # The delay in percentages
            postpone_start=True  # For sequential or while activities 
        )

        # We simply repeat the activity for demonstration purposes
        sequence = model.SequentialActivity(
            env=self.env,
            name='sequential activity',
            sub_processes=[activity, activity, activity],
            registry={}
        )

        return [activity]

    def execute_simulation(self):
        """ Function starts the simulation. """

        self.activities = self.define_activities()
        self.env.run()
        return 'SUCCESSFUL'

    @property
    def event_log(self):
        """ Function returns the event log. """
        return get_event_log(activity_list=self.activities)

    @property
    def project_length(self):
        """ Function returns the project length. """
        log = self.event_log
        dt = (log.Timestamp.iloc[-1] - log.Timestamp.iloc[0])
        return dt.total_seconds() / (3600)


# Define the delayed basic activity using inheritance.
class DelayedBasicActivity(plugins.HasDelayPlugin, model.BasicActivity):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

In [59]:
sim = SimulationEnvironment(datetime.datetime(2021, 1, 1))
res = sim.execute_simulation()
log = sim.event_log
log

,Timestamp,ActivityState,Description,ActivityID
0,2021-01-01 00:00:00.000000,START,Delayed basic activity,32671475-737c-495b-97e5-3040537c66ca
1,2021-01-01 05:00:00.000000,STOP,Delayed basic activity,32671475-737c-495b-97e5-3040537c66ca
2,2021-01-01 05:00:00.000000,WAIT_START,Delayed basic activity,32671475-737c-495b-97e5-3040537c66ca
3,2021-01-01 05:00:08.424215,WAIT_STOP,Delayed basic activity,32671475-737c-495b-97e5-3040537c66ca
4,2021-01-01 05:00:08.424215,START,Delayed basic activity,32671475-737c-495b-97e5-3040537c66ca
5,2021-01-01 10:00:08.424215,STOP,Delayed basic activity,32671475-737c-495b-97e5-3040537c66ca
6,2021-01-01 10:00:08.424215,WAIT_START,Delayed basic activity,32671475-737c-495b-97e5-3040537c66ca
7,2021-01-01 10:01:02.497284,WAIT_STOP,Delayed basic activity,32671475-737c-495b-97e5-3040537c66ca
8,2021-01-01 10:01:02.497284,START,Delayed basic activity,32671475-737c-495b-97e5-3040537c66ca
9,2021-01-01 15:01:02.497284,STOP,Delayed basic activity,32671475-737c-495b-97e5-3040537c66ca


## PROOF THAT DELAY LENGTH IS ACTUALLY VARYING

In [60]:
import plotly.graph_objects as go

In [61]:
periods = []

for i in range(1000):
    sim = SimulationEnvironment(datetime.datetime(2021, 1, 1))
    res = sim.execute_simulation()
    periods.append(sim.project_length)

In [62]:
hist = go.Histogram(
    x=periods, 
    autobinx=True, 
    marker=dict(
        color='red',
        opacity=0.75,
        line=dict(color='black', width=1)
    ),
    cumulative=dict(enabled=True),
    histnorm='probability'
)

fig = go.Figure(data=hist)

fig.update_layout(
    xaxis_title = 'Estimated Project Length [hrs]',
    yaxis_title = 'Probability [-]',
    title='Project Length Distribution for 1000 simulations'
)

fig.show()

<div style="width:500px;">

Note how the above image transforms to a normal distribution, whilst the delay was described using an uniform distribution. This is due to the central limit theorem, which establishes that, in many situations, when independent random variables are added their sum tends toward a normal distribution. If the `sub_processes` in the `model.SequentialActivity` contains only a single activity, we would see the uniform distribution!

</div>